# TP5-6 : Problème (P1) - contraintes de précédence

Initialization (à décommenter et exécuter une seule fois)

In [1]:
import Pkg; Pkg.add("Clp")

   Updating registry at `C:\Users\mlg15\.julia\registries\General`
  Resolving package versions...
  Installed Clp ─ v0.8.3
Updating `C:\Users\mlg15\.julia\environments\v1.5\Project.toml`
  [e2554f3b] + Clp v0.8.3
Updating `C:\Users\mlg15\.julia\environments\v1.5\Manifest.toml`
  [e2554f3b] + Clp v0.8.3
   Building Clp → `C:\Users\mlg15\.julia\packages\Clp\NUuKQ\deps\build.log`


Récupération des données

In [1]:
D=[2, 3, 1, 4, 1]

5-element Array{Int64,1}:
 2
 3
 1
 4
 1

Modélisation et résolution de (P1) à l'aide d'un programme linéaire

In [2]:
using JuMP
using Clp

model = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model, t[i in 1:5] >= 0)
@variable(model, tfin)

# define objective function
@objective(model, Min, tfin)

# define constraints: t_j - t_i  >= D[i], \forall i predecesseur de j
@constraint(model, t[2] - t[1] >= D[1] )
@constraint(model, t[3] - t[1] >= D[1] )
@constraint(model, t[4] - t[2] >= D[2] )
@constraint(model, t[4] - t[3] >= D[3] )
@constraint(model, t[5] - t[3] >= D[3] )

#define constraints: tfin - t_i >= Duree[i], \forall i predecesseur de j
@constraint(model, tfin - t[4] >= D[4] )
@constraint(model, tfin - t[5] >= D[5] )

println(model)

print("start solve ... ")
optimize!(model)
print("... end solve")


println("\n\nSolution PL:\n \t t=", value.(t), "\t tfin=", value(tfin))


Min tfin
Subject to
 -t[1] + t[2] >= 2.0
 -t[1] + t[3] >= 2.0
 -t[2] + t[4] >= 3.0
 -t[3] + t[4] >= 1.0
 -t[3] + t[5] >= 1.0
 -t[4] + tfin >= 4.0
 -t[5] + tfin >= 1.0
 t[1] >= 0.0
 t[2] >= 0.0
 t[3] >= 0.0
 t[4] >= 0.0
 t[5] >= 0.0

start solve ... ... end solve

Solution PL:
 	 t=[0.0, 2.0, 2.0, 5.0, 8.0]	 tfin=9.0


In [3]:
using JuMP
using Clp

model = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model, t[i in 1:2] >= 0)
@variable(model, tfin)

# define objective function
@objective(model, Min, tfin)

# define constraints: t_j - t_i  >= D[i], \forall i predecesseur de j
@constraint(model, t[2] - t[1] >= D[1] )
@constraint(model, t[3] - t[1] >= D[1] )
@constraint(model, t[4] - t[2] >= D[2] )
@constraint(model, t[4] - t[3] >= D[3] )
@constraint(model, t[5] - t[3] >= D[3] )

#define constraints: tfin - t_i >= Duree[i], \forall i predecesseur de j
@constraint(model, tfin - t[4] >= D[4] )
@constraint(model, tfin - t[5] >= D[5] )

println(model)

print("start solve ... ")
optimize!(model)
print("... end solve")


println("\n\nSolution PL:\n \t t=", value.(t), "\t tfin=", value(tfin))


LoadError: BoundsError: attempt to access 2-element Array{VariableRef,1} at index [3]

In [4]:
import Pkg
using LinearAlgebra

# Fonction calculant le maximum d'un tableau
function maximumTableau(tableau)
    taille = size(tableau)[1]
    maximum = tableau[1]
    indice = 1
    for i in 1:taille
          if (tableau[i] > maximum)
            maximum = tableau[i]
            indice = i
        end
    end
    return maximum, indice
end

# Fonction retournant la liste des prédcesseurs d'un noeud dans un graphe
function Predecesseur(graphe,j)
    nbSommets = size(graphe)[1]
    listePredecesseurs = []
    for i in 1:nbSommets
        # Si le noeud a une valeur autre que 0, Inf ou - Inf, on l'ajoute 
        # à la liste des prédécesseurs du noeud dont on recherche les 
        # prédécesseurs
        if graphe[i,j]!= Inf && graphe[i,j]!= -Inf
            listePredecesseurs = append!([i],listePredecesseurs)
        end
    end
    return listePredecesseurs
end

function BellmanFordLong(graphe, source)
    
    # Nombre de sommets du graphe
    nbSommets = size(graphe)[1]

    # Distance parcourue pour chaque
    distance =  fill(-Inf,(nbSommets + 1,nbSommets))
    distance[1, source]=0

    k=1
    bool=true
    
    # L'algorithme de Bellman-Ford s'arrête dès que 2 lignes du tableau de
    # suite sont égales ou que k vaut nbSommets + 1
    while (bool && k<nbSommets+1)
        # On incrémente k
        k += 1
        # La ligne k est initialisée à partie de la ligne k-1
        distance[k, :] = distance[k-1, :]
        for i in 2:nbSommets
            # On récupère les prédécesseurs de i
            prede_i= Predecesseur(graphe,i)
            l=[]
            for j in prede_i
                # on ajoute la somme du poids du predecesseur et de sa distance
                l=append!([distance[k-1,j]+graphe[j,i]],l)
            end
            # On récupère le maximum de la liste précédente
            distance[k,i], indice = maximumTableau(l)
        end
        # Si cette ligne du tableau est la même que la précédente on s'arrête
        if (distance[k-1, :]==distance[k, :])
                bool=false
        end
    end
    # On retourne la dernière ligne
    return distance[k,:]
end

graphe = 
# debut A   B    C    D    E    Fin
[  0    0   -Inf -Inf -Inf -Inf -Inf
   -Inf 0    2   2   -Inf -Inf -Inf;
  -Inf  -Inf 0    -Inf 3   -Inf -Inf;
  -Inf  -Inf -Inf  0   1   1    -Inf;
  -Inf  -Inf -Inf -Inf 0   -Inf  4;
  -Inf  -Inf -Inf -Inf -Inf  0   1;
  -Inf  -Inf -Inf -Inf -Inf -Inf  0;
]

distance = BellmanFordLong(graphe, 1)
println("Date de début de [t1,t2,t3,t4,t5]: ", distance[2:length(distance) - 1])
println("Date de fin : ", distance[length(distance)])

Date de début de [t1,t2,t3,t4,t5]: [0.0, 2.0, 2.0, 5.0, 3.0]
Date de fin : 9.0
